# API Exercises

Here are two exercises that involve working with APIs and dictionaries.

One is using the Open Brewery API found at https://www.openbrewerydb.org/, and the other is using the API for UK Police Data, found at https://data.police.uk/docs/.

You can complete them in either order!

Remember that you can create new cells with esc + a or b

## Breweries

### Q1: Load the first page of results with 50 results per page

In [2]:
import requests
import json
import pandas as pd

In [3]:
url = 'https://api.openbrewerydb.org/breweries?page=1&per_page=50'
response = requests.get(url).json()
len(response)


50

In [4]:
df1=pd.DataFrame(response)

### Q2: This is only the first 50 results.  Get the next 50 and put them together.

In [5]:
url = 'https://api.openbrewerydb.org/breweries?page=2&per_page=50'
response2 = requests.get(url).json()
len(response2)


50

In [6]:
df2=pd.DataFrame(response2)

In [7]:
df=pd.concat([df1,df2])

### Q3: How many of these 100 breweries in are in Alaska?

In [8]:
len(df[df.state=='Alaska'])

3

### Q4: Of these 100 breweries, what are the different unique brewery types?

In [9]:
df.brewery_type.unique()


array(['micro', 'contract', 'brewpub', 'regional', 'planning',
       'proprietor'], dtype=object)

In [10]:
df.head()

,brewery_type,city,country,id,latitude,longitude,name,phone,postal_code,state,street,tag_list,updated_at,website_url
0,micro,Birmingham,United States,2,33.524521,-86.774322,Avondale Brewing Co,2057775456,35222-1932,Alabama,201 41st St S,[],2018-08-23T23:19:57.825Z,http://www.avondalebrewing.com
1,micro,Tuscaloosa,United States,4,33.1984907123707,-87.5621551272424,Band of Brothers Brewing Company,2052665137,35401-4653,Alabama,1605 23rd Ave,[],2018-08-23T23:19:59.462Z,http://www.bandofbrosbrewing.com
2,micro,Birmingham,United States,44,33.5128492349817,-86.7914000624146,Trim Tab Brewing,2057030536,35233-3401,Alabama,2721 5th Ave S,[],2018-08-23T23:20:31.423Z,http://www.trimtabbrewing.com
3,micro,Huntsville,United States,46,34.7277523,-86.5932014,Yellowhammer Brewery,2569755950,35805-3046,Alabama,2600 Clinton Ave W,[],2018-08-23T23:20:33.102Z,http://www.yellowhammerbrewery.com
4,micro,Wasilla,United States,55,61.5752695,-149.4127103,Bearpaw River Brewing Co,,99654-7679,Alaska,4605 E Palmer Wasilla Hwy,[],2018-08-23T23:20:40.743Z,http://bearpawriverbrewing.com


### Q5: What is the cloest brewery to "Devil's Potion Brewing Company LLC" ?
* Hint 1: Use Euclidian distance w/ longitude and latitude (assume longitude and latitude are a Carteisan coordinate system)
* Hint 2: You'll have to ignore the entries with "none" for latitude or longitude

In [11]:
def closest_brewery(name):
    cl=df[df.name!=name]
    lat=float(df[df.name==name].latitude.min())
    lon=float(df[df.name==name].longitude.min())
    cl.fillna(99999)
    cl['dis']= (cl.latitude.astype(float) - lat)**2 + (cl.longitude.astype(float) - lon)**2
    cl['dis']= cl['dis']**.5 
    cl=cl.sort_values(by=['dis'])
    


    return cl.iloc[0]
    
    


In [12]:
closest_brewery("Devil's Potion Brewing Company LLC")

/opt/conda/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


brewery_type                               micro
city                                  San Marcos
country                            United States
id                                           924
latitude                               33.141537
longitude                            -117.149141
name            Port Brewing Co / The Lost Abbey
phone                                 8009186816
postal_code                           92069-2983
state                                 California
street                      155 Mata Way Ste 104
tag_list                                      []
updated_at              2018-08-24T00:01:07.397Z
website_url           http://www.portbrewing.com
dis                                    0.0705113
Name: 23, dtype: object

### Q6: Write a function to find the closest brewery to any other given brewery

### Q7: How would you get the first 10 pages from this API and put them all together using a for loop?

In [13]:
pag=range(1,11)
dfs=[]
for p in pag:
    url = 'https://api.openbrewerydb.org/breweries?page=' + str(p) +'&per_page=50'
    js= requests.get(url).json()
    df= pd.DataFrame(response)
    dfs.append(df)
    



# Crime in the UK

### We will be analyzing different crimes reported in the UK as provided by https://data.police.uk/docs/

# Exploratory analysis
##### 1. How many total crimes were there at latitude : 52.63902 and -1.131321 on November of 2017.
Use the street level crimes data, the documentation for the API can be found at https://data.police.uk/docs/method/crime-street/

In [23]:
parameters={'lat': 52.63902, 'lng': -1.131321, 'date': (2017-11)}

url= 'https://data.police.uk/docs/method/crime-street/'
    
response = requests.get(url, params=parameters)



In [28]:
response.json()

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

##### 2. We've queried the API once, but it could get annoying to retype the url over and over again, create a function `make_api_request` that enables you to query the API.


 The parameters for the function should be:
* lat (float) : latitude
* lng (float) : longitude
* date (string): Date in the format YYYY-MM
    * default value = `None`
    
And it should return a json object of 

for more information on default values check out http://blog.thedigitalcatonline.com/blog/2015/02/11/default-arguments-in-python/

##### 3. Write a function `categories_of_crime` that will determine the count of each type of crime for a given latitude and longitude. This is labelled as 'category' in the records. Your function should call the `make_api_request` function you created.

The parameters for the function should be:

* lat (float) : latitude
* lng (float) : longitude
* date (str) default = None

The function should return:
* a dictionary with the count of each type of crime



Once you've created the function, try it with these locations
* lat, lng of 51.5017861,-0.1432319   (Buckingham Palace)
* lat, lng of 53.480161, -2.245163     (Manchester)

**Bonus**: 
* Write a function that determines the difference between Buckingham Palace and Manchester in terms of the number of crimes in each category.
    * In which category is there the largest absolute difference between the category of crime?
* Create a histogram depiction of the categories of crime

##### 4. Create a function `find_outcome_statuses` that will determine outcome statuses for a given latitude and longitude and date (optional)?
Investigate the data to determine where the outcome statuses are located.

**NOTE**: You'll notice that some of these crimes do not have crime outcomes. Make these into the category of "Not Resolved."

**NOTE 2**: These might take a long time to execute if you do not specify a month

**Bonus**: What is the ratio of crimes investigated to those not investigated? Is it higher near London or Manchester?

##### 5. Write a function `month_highest_crimes` that will return the month that had the highest number of crimes for a latitude, longitude and a year.

Inputs
* lat (float) : latitude
* lng (float) : longitude
* year (str) : in the format YYYY

Output
* month with highest crime (int)

**Bonus** Make a graph of how the number of crimes changed over time for a year. This will likely require a new function. Is seasonality a factor? Do the type of crimes change over time?

### Bonus Open Ended Questions

1. Take a look at the https://data.police.uk/docs/method/stops-street/ API. Is there a correlation between gender and being stopped and searched? How about race and being stopped and searched?